In [1]:
from pathlib import Path
from io import StringIO
import numpy as np
import pandas as pd

from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from sklearn.metrics import (
    precision_score,
    recall_score,
    f1_score,
    precision_recall_curve,
)

from util import (
    make_intervals,
    compute_event_wise_metrics,
    predict_with_PCA,
)

# Prepare original data
Original training data comes with the following header and lengthy column names:

```
Created: 10/9/2017 6:05:57.359 PM Malay Peninsula Standard Time                       
Number of rows: 1.2096E+6
Interpolation interval: 1 seconds

```

Please put the `prepare_WADI.sh` script in the same directory as the data and run it. Then check the checksum of processed files by running the next two cells.

In [2]:
!md5sum ./data/WADI/*.csv

8d3d8b91aaf79cf845ed94fc117f3849  ./data/WADI/WADI_14days.csv
649ec203ad98ffa2073a91e5ad51ed5e  ./data/WADI/WADI_A1_2017_attacks.csv
2d6a4d2a44e085cd97e5e6e2dd4f7a3c  ./data/WADI/WADI_attackdata.csv


In [3]:
!sha1sum ./data/WADI/*.csv

0d35aa64237db1c2e23379a9b995dafc636807cb  ./data/WADI/WADI_14days.csv
2a1eea939e6d7b0ecaf26643afa10c720927a12e  ./data/WADI/WADI_A1_2017_attacks.csv
c45b39752207639ac57a8cce88f0652e148203c8  ./data/WADI/WADI_attackdata.csv


# Load data (csv)

In [4]:
DATADIR = Path("./data/WADI")

normal = pd.read_csv(
    DATADIR / "WADI_14days.csv",
    parse_dates={"Timestamp": ["Date", "Time"]},
    date_format="%m/%d/%Y %I:%M:%S.000 %p",
    index_col="Timestamp"
)

attack = pd.read_csv(
    DATADIR / "WADI_attackdata.csv",
    parse_dates={"Timestamp": ["Date", "Time"]},
    date_format="%m/%d/%Y %I:%M:%S.000 %p",
    index_col="Timestamp"
)

attacks_ts = pd.read_csv(
    DATADIR / "WADI_A1_2017_attacks.csv", parse_dates=["StartTime", "EndTime"],
    date_format="%d/%m/%Y %H:%M:%S",
)

In [5]:
assert (normal.columns == attack.columns).all()

# Make Labels

Make labels using attacks' timestamps.

In [6]:
y_true_ts = np.zeros(len(attack))
gt_intervals = []
index = list(attack.index)
for _, (onset, offset, *_) in attacks_ts.iterrows():
    onset = index.index(onset)
    offset = index.index(offset) + 1
    y_true_ts[onset:offset] = 1
    gt_intervals.append((onset, offset))
y_true_ts.mean()
y_true = y_true_ts == 1

print("Contamination rate:", f"{y_true.mean()*100:.2f}")
print("Number of anomalous events:", len(gt_intervals))
event_lengths = np.diff(gt_intervals).reshape(-1)
print("Min event length:", np.min(event_lengths))
print("Max event length:", np.max(event_lengths))
print("Average event length:", round(np.mean(event_lengths)))
print("Median event length:", round(np.median(event_lengths)))

Contamination rate: 5.71
Number of anomalous events: 14
Min event length: 88
Max event length: 1741
Average event length: 711
Median event length: 630


# Hyperparameters

In [7]:
SEED = 1234

MIN_SCALE = -1.
MAX_SCALE = 1.

MIN_CLIP = -5.
MAX_CLIP = 5.

PCA_N_COMP = 46 # other vlaues may also work as well
SCORE_AGG_FN = np.mean
SCORE_SMOOTHING_HISTORY_SIZE = 30

# Algorithm

In [8]:
drop_cols = [
    "Row",
    "2_LS_001_AL",
    "2_LS_002_AL",
    "2_P_001_STATUS",
    "2_P_002_STATUS",
]
X_train = (
    normal.drop(columns=drop_cols).ffill().to_numpy().astype(np.float32)
)
X_test = (
    attack.drop(columns=drop_cols).ffill().to_numpy().astype(np.float32)
)

print(X_train.shape)

scaler = MinMaxScaler((MIN_SCALE, MAX_SCALE))
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

(1209601, 123)


In [9]:
np.random.seed(SEED)

pca = PCA(n_components=PCA_N_COMP)
pca.fit(X_train)

residual = predict_with_PCA(
    pca,
    X_test.clip(min=MIN_CLIP, max=MAX_CLIP),
    score_agg_fn=np.mean,
    smooth_n=SCORE_SMOOTHING_HISTORY_SIZE,
)
scores = residual

# Evaluation

## Point-wise F1 score
We first compute the best point-wise F1 score, then we use the corresponding threshold to compute other metrics.

In [10]:
precision, recall, thresholds = precision_recall_curve(y_true, scores)

f1_scores = 2 * recall * precision / (recall + precision + 1e-10)
round(f1_scores.max(), 4)

0.3744

## Event-wise metrics
For these we use the threshold that yields the best **point-wise** F1 score.

In [11]:
argmax = f1_scores.argmax()
best_threshold = thresholds[argmax]

In [12]:
y_pred = scores >= best_threshold


TP_ew, FP_ew, FN_ew, P_ew, R_ew, F1_ew, F1_c = compute_event_wise_metrics(
    y_true, y_pred, gt_intervals
)

print("Point-wise F1:\t\t", round(f1_score(y_true, y_pred), 4))
print("Event-wise F1 (F1_ew):\t", round(F1_ew, 4))
print("Composite F1 (F1_c):\t", round(F1_c, 4))
print("True positive events:\t", TP_ew)
print("False positive events:\t", FP_ew)


Point-wise F1:		 0.3744
Event-wise F1 (F1_ew):	 0.6085
Composite F1 (F1_c):	 0.6546
True positive events:	 7
False positive events:	 2


## Better composite F1 score with a smaller smoothing window
A better **composite F1** score can be achieved by setting `SCORE_SMOOTHING_HISTORY_SIZE` to `15` instead of `30`. This will have little impact on the **point-wise F1**, but the **event-wise** score will decrease due to a higher number of false alarms at the event level.


Please update `SCORE_SMOOTHING_HISTORY_SIZE` above and run all cells starting from **Hyperparameters**:

In [19]:
y_pred = scores >= best_threshold


TP_ew, FP_ew, FN_ew, P_ew, R_ew, F1_ew, F1_c = compute_event_wise_metrics(
    y_true, y_pred, gt_intervals
)

print("Point-wise F1:\t\t", round(f1_score(y_true, y_pred), 4))
print("Event-wise F1 (F1_ew):\t", round(F1_ew, 4))
print("Composite F1 (F1_c):\t", round(F1_c, 4))
print("True positive events:\t", TP_ew)
print("False positive events:\t", FP_ew)

Point-wise F1:		 0.3825
Event-wise F1 (F1_ew):	 0.4704
Composite F1 (F1_c):	 0.7122
True positive events:	 8
False positive events:	 12
